In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

# Loading Datasets

In [24]:
labels = ['Lagged', 'MA', 'WMA', 'MA-Lagged', 'WMA-Lagged']

def load_datasets():
    datasets = dict()
    for lb in labels:
        new_df = pd.read_excel(f"River-Data-{lb}.xlsx")
        new_df.drop(["Unnamed: 0"], axis=1, inplace=True)
        datasets[lb] = new_df
    
    return datasets

data = load_datasets()

# Utility Functions

In [25]:
# Functions for standardising and unstandardising columns
def standardise_columns(df, cols):
    subset_df = df[cols]
    subset_df = 0.8 * ((subset_df - subset_df.min()) / (subset_df.max() - subset_df.min())) + 0.1
    return subset_df

def unstandardise_columns(df, cols, max_val, min_val):
    subset_df = df[cols]
    subset_df = ((subset_df - subset_df.min()) / 0.8) * (max_val - min_val) + min_val
    return subset_df

def standardise_value(x, max_val, min_val):
    return 0.8 * ((x - min_val)) / (max_val - min_val) + 0.1

def unstandardise_value(x, max_val, min_val):
    return ((x - 0.1) / 0.8) * (max_val - min_val) + min_val

# ANN Class

In [26]:
class BasicAnn:
    def __init__(self, layers, max_st_val, min_st_val, activ_func="sigmoid"):
        self.layers = layers
        self.num_layers = len(layers)
        self.max_val = max_st_val
        self.min_val = min_st_val
        self.activ_func = activ_func
        
        weight_shapes = [(layers[i-1],layers[i]) for i in range(1, len(layers))]
        self.weights = {
            f"W{i+1}": np.random.standard_normal(s)/s[0]**0.5 
            for i, s in enumerate(weight_shapes) 
        }
        self.biases = {
            f"B{i+1}": np.random.randn(l,1)/l**0.5 
            for i, l in enumerate(layers[1:])
        }
    
    def activation(self, x):
        if self.activ_func == "sigmoid":
            return 1/(1+np.exp(-x))
        elif self.activ_func == "tanh":
            return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        elif self.activ_func == "relu":
            return x * (x > 0)
        elif self.activ_func == "linear":
            return x
    
    def activation_deriv(self, a):
        if self.activ_func == "sigmoid":
            return a * (1 - a)
        elif self.activ_func == "tanh":
            return 1 - a**2
        elif self.activ_func == "relu":
            return 1 * (a > 0)
        elif self.activ_func == "linear":
            return np.ones(a.shape)
    
    def train(self, features, targets, epochs=1000, learning_rate=0.1):
        results = pd.DataFrame()
        real_targets = unstandardise_value(targets, self.max_val, self.min_val)
        num_targets = len(targets)
        
        for _ in range(epochs):
            # Forward pass
            activations = self.forward_pass(features)

            # Error calculation
            output_layer = activations[f"A{self.num_layers - 1}"]
            real_preds = unstandardise_value(output_layer, self.max_val, self.min_val)
            results = results.append({
                "mse": mean_squared_error(real_targets, real_preds),
                "rmse": mean_squared_error(real_targets, real_preds, squared=False),
                "mae": mean_absolute_error(real_targets, real_preds),
                "r_sqr": r2_score(real_targets, real_preds),
                "st_mse": mean_squared_error(targets, output_layer),
                "st_rmse": mean_squared_error(targets, output_layer, squared=False),
                "st_mae": mean_absolute_error(targets, output_layer),
                "st_r_sqr": r2_score(targets, output_layer)
            }, ignore_index=True)

            # Backward pass
            deltas = self.compute_deltas(activations, targets, output_layer)
            self.update_weights(deltas, activations, features, num_targets, learning_rate)
        
        return results
    
    def predict(self, test_inputs, st_actual_outputs):
        # Forward pass
        activations = self.forward_pass(test_inputs)
        st_preds = activations[f"A{self.num_layers - 1}"]
        
        actual_outputs = unstandardise_value(st_actual_outputs, self.max_val, self.min_val)
        preds = unstandardise_value(st_preds, self.max_val, self.min_val)
        
        results = pd.DataFrame(
            data={
                "Actual Values": actual_outputs.flatten(), 
                "Predicted Values": preds.flatten(),
                "Actual Values (Standardised)": st_actual_outputs.flatten(),
                "Predicted Values (Standardised)": st_preds.flatten(),
            }
        )
        
        results["Absolute Error"] = abs(results["Actual Values"] - results["Predicted Values"])
        results["Absolute Error (Standardised Values)"] = abs(results["Actual Values (Standardised)"] - results["Predicted Values (Standardised)"])
        
        error_metrics = pd.DataFrame(data={
            "mse": [mean_squared_error(actual_outputs, preds)],
            "rmse": [mean_squared_error(actual_outputs, preds, squared=False)],
            "mae": [mean_absolute_error(actual_outputs, preds)],
            "r_sqr": [r2_score(actual_outputs, preds)],
            "st_mse": [mean_squared_error(st_actual_outputs, st_preds)],
            "st_rmse": [mean_squared_error(st_actual_outputs, st_preds, squared=False)],
            "st_mae": [mean_absolute_error(st_actual_outputs, st_preds)],
            "st_r_sqr": [r2_score(st_actual_outputs, st_preds)]
        })
        
        return results, error_metrics
    
    def forward_pass(self, features):
        activation = self.activation(np.dot(features, self.weights["W1"]) + self.biases["B1"].T)
        activations = {"A1": activation}
        for i in range(2, self.num_layers):
            activation = self.activation(np.dot(activation, self.weights[f"W{i}"]) + self.biases[f"B{i}"].T)
            activations[f"A{i}"] = activation
        
        return activations
    
    def compute_deltas(self, activations, targets, output_layer):
        ## Computing deltas
        output_err = targets - output_layer
        output_delta = output_err * self.activation_deriv(output_layer)
        deltas = {"dw1": output_delta}

        for i in range(self.num_layers - 1, 1, -1):
            dw = deltas[f"dw{self.num_layers - i}"]
            act = activations[f"A{i-1}"]
            w = self.weights[f"W{i}"]
            deltas[f"dw{self.num_layers - i + 1}"] = np.dot(dw, w.T) * self.activation_deriv(act)
        
        return deltas
    
    def update_weights(self, deltas, activations, features, num_targets, l_rate):
        ## Updating weights and biases
        delta = deltas[f"dw{self.num_layers - 1}"]
        self.weights["W1"] += l_rate * (np.dot(features.T, delta)) / num_targets
        self.biases["B1"] += l_rate * (np.dot(delta.T, np.ones((num_targets, 1)))) / num_targets

        for i in range(2, self.num_layers):
            act = activations[f"A{i-1}"]
            dw = deltas[f"dw{self.num_layers - i}"]
            self.weights[f"W{i}"] += l_rate * (np.dot(act.T, dw)) / num_targets
            self.biases[f"B{i}"] += l_rate * np.dot(dw.T, np.ones((num_targets, 1))) / num_targets      

#### Build, Train and Test ANN Model

In [27]:
def build_train_test(feature_set, feature_cols, target_cols, layers=("auto", 1), activ_func="linear"):
    # Getting standardisation values for targets
    min_val = feature_set[target_cols].min()[0]
    max_val = feature_set[target_cols].max()[0]
    
    # Standardising feature set
    st_feature_set = standardise_columns(feature_set, feature_set.columns)
    features = st_feature_set[feature_cols]
    targets = st_feature_set[target_cols]
    
    # Splitting data
    X_train, X_test, y_train, y_test = train_test_split(features, targets)
    
    # Building model
    if layers[0] == "auto":
        layers = (len(feature_cols),) + layers[1:]
    
    ann = BasicAnn(layers, max_val, min_val, activ_func)
    
    # Training model
    training_results = ann.train(X_train.to_numpy(), y_train.to_numpy())
    print(training_results)
    print("Final error metrics:\n\n", training_results.iloc[-1], end="\n\n")
    training_results.mae.plot()
    
    # Predicting model
    prediction_results = ann.predict(X_test.to_numpy(), y_test.to_numpy())
    predictions, error_metrics = prediction_results[0], prediction_results[1]
    print(prediction_results)
    print(error_metrics)
    predictions.plot(y=["Actual Values", "Predicted Values"])
    
    return {
        "training_results": training_results,
        "predictions": predictions,
        "error_metrics": error_metrics
    }

# Experiment 1: Feature Selection

#### Building Feature Sets

In [28]:
# Function for building custom feature and target sets
def build_feature_set(*datasets):
    assert len(datasets) > 0, "No data sets entered"
    datasets = list(datasets)
    min_rows = min(d.shape[0] for d in datasets)
    
    for i, ds in enumerate(datasets):
        datasets[i] = ds.truncate(before=ds.shape[0]-min_rows).reset_index()
        datasets[i].drop(["index"], axis=1, inplace=True)
        
    merged_df = datasets[0].iloc[:, :2]
    for ds in datasets:
        merged_df = pd.concat([merged_df, ds.iloc[:, 2:]], axis=1)
    
    merged_cols = list(merged_df.columns)
    selected_cols = []
    
    for i in range(0, len(merged_cols), 2):
        format_str = f"{i+1}) {merged_cols[i]}"
        if i != len(merged_cols) - 1:
            second_part = f"{i+2}) {merged_cols[i+1]}"
            num_spaces = 50 - len(format_str)
            format_str += num_spaces*" " + second_part
        print(format_str)
    
    selected_indices = input("\nSelect columns: ")
    for index in selected_indices.split(","):
        if "-" in index:
            first_i, second_i = index.split("-")
            selected_cols += merged_cols[int(first_i) - 1: int(second_i)]
        else:
            selected_cols.append(merged_cols[int(index) - 1])
    
    return merged_df[selected_cols]

#### Correlations Between Features

In [29]:
# Utility function for plotting a correlation heatmap of a given feature set
def plot_correlation_matrix(corr_data, title, figsize=(16,6), mask=False):
    if mask:
        mask = np.triu(np.ones_like(corr_data, dtype=bool))
    plt.figure(figsize=figsize, dpi=500)
    heatmap = sns.heatmap(corr_data, vmin=-1, vmax=1, annot=True, mask=mask)
    heatmap.set_title(title)
    plt.show()

In [30]:
# fs = build_feature_set(data['WMA-Lagged'], data['WMA'], data['Lagged'], data['MA'], data['MA-Lagged'])
fs = build_feature_set(data['Lagged'])

1) Date                                           2) Skelton MDF (Cumecs)
3) Crakehill MDF (t-1)                            4) Skip Bridge MDF (t-1)
5) Westwick MDF (t-1)                             6) Skelton MDF (t-1)
7) Crakehill MDF (t-2)                            8) Skip Bridge MDF (t-2)
9) Westwick MDF (t-2)                             10) Skelton MDF (t-2)
11) Crakehill MDF (t-3)                           12) Skip Bridge MDF (t-3)
13) Westwick MDF (t-3)                            14) Skelton MDF (t-3)
15) Arkengarthdale DRT (t-1)                      16) East Cowton DRT (t-1)
17) Malham Tarn DRT (t-1)                         18) Snaizeholme DRT (t-1)
19) Arkengarthdale DRT (t-2)                      20) East Cowton DRT (t-2)
21) Malham Tarn DRT (t-2)                         22) Snaizeholme DRT (t-2)
23) Arkengarthdale DRT (t-3)                      24) East Cowton DRT (t-3)
25) Malham Tarn DRT (t-3)                         26) Snaizeholme DRT (t-3)

Select columns: 2,3-6,15-26


In [31]:
fs

,Skelton MDF (Cumecs),Crakehill MDF (t-1),Skip Bridge MDF (t-1),Westwick MDF (t-1),Skelton MDF (t-1),Arkengarthdale DRT (t-1),East Cowton DRT (t-1),Malham Tarn DRT (t-1),Snaizeholme DRT (t-1),Arkengarthdale DRT (t-2),East Cowton DRT (t-2),Malham Tarn DRT (t-2),Snaizeholme DRT (t-2),Arkengarthdale DRT (t-3),East Cowton DRT (t-3),Malham Tarn DRT (t-3),Snaizeholme DRT (t-3)
0,23.47,9.46,4.124,8.057,23.60,0.0,0.0,0.8,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,4.0
1,60.70,9.41,4.363,7.925,23.47,2.4,24.8,0.8,61.6,0.0,0.0,0.8,0.0,0.0,0.0,0.8,0.0
2,98.01,26.30,11.962,58.704,60.70,11.2,5.6,33.6,111.2,2.4,24.8,0.8,61.6,0.0,0.0,0.8,0.0
3,56.99,32.10,10.237,34.416,98.01,0.0,0.0,1.6,0.8,11.2,5.6,33.6,111.2,2.4,24.8,0.8,61.6
4,56.66,19.30,7.254,22.263,56.99,5.6,4.0,17.6,36.0,0.0,0.0,1.6,0.8,11.2,5.6,33.6,111.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,29.52,11.70,6.075,12.671,33.06,0.0,0.8,0.0,0.0,0.0,3.2,0.8,1.6,0.0,0.0,0.0,0.0
1444,28.67,10.90,5.721,11.558,29.52,1.6,14.4,8.8,3.2,0.0,0.8,0.0,0.0,0.0,3.2,0.8,1.6
1445,29.31,11.10,5.486,11.411,28.67,11.2,11.2,4.8,4.8,1.6,14.4,8.8,3.2,0.0,0.8,0.0,0.0
1446,34.28,12.10,5.329,11.781,29.31,3.2,4.8,0.0,0.8,11.2,11.2,4.8,4.8,1.6,14.4,8.8,3.2


In [39]:
target_cols = [fs.columns[0]]
feature_cols = list(fs.columns[1:])

build_train_test(fs, feature_cols, target_cols, layers=("auto", 3, 1), activ_func="sigmoid")

NameError: name 'MultiLayerAnn' is not defined